In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt



# CRUD Python module file name and class name
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# username and password for database (AAC)
# server (host2)and port settings (port3)

username = "aacuser"
password = "over9000"
host2 = "nv-desktop-services.apporto.com"
port3 = 30238


# Connect to database via CRUD Module
db = AnimalShelter(username, password, host2, port3)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#Sort Dataframe by rec_num before passing to DataTable
df = df.sort_values(by='rec_num')

#Limit the number of columns (cordinates are eliminated)
#df = df.iloc[:, :16]

# Loads and Displays Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Hr(),
    html.Img(
        src='data:image/png;base64,{}'.format(encoded_image.decode()), 
        style={
            'display': 'block',
            'margin-left': 'auto',
            'margin-right': 'auto',
            'width': '10%',  # Reduces the image size
            'height': 'auto'
        }
    ),
    html.Br(),
    

# Radio Button Filtering Options
    
dcc.RadioItems(
    id='filter-type',
    options=[
        {'label': 'Water Rescue', 'value': 'water-rescue'},
        {'label': 'Mountain Rescue', 'value': 'mountain-rescue'},
        {'label': 'Disaster Rescue', 'value': 'disaster-rescue'},
        {'label': 'Reset', 'value': 'reset'}
    ],
    value='reset',
    labelStyle={'display': 'inline-block', 'margin-right': '10px'}
),
    html.Hr(),
    
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        page_size=10,
        row_selectable="single",  
        sort_action="native",
        filter_action="native",  # Adds filtering capability
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(
    [Output('datatable-id', 'data'),
     Output('datatable-id', 'columns')],
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    # Initialize an empty DataFrame
    df = pd.DataFrame()

    # Will query the MongoDB based on the filter selection
    if filter_type == 'water-rescue':
        df = pd.DataFrame.from_records(db.read({
            "animal_type": "Dog",
            "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26.0, "$lte": 156.0}
        }))
    elif filter_type == 'mountain-rescue':
        df = pd.DataFrame.from_records(db.read({
            "animal_type": "Dog",
            "breed": {"$in": ["German Shepard", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26.0, "$lte": 156.0}
        }))
    elif filter_type == 'disaster-rescue':
        df = pd.DataFrame.from_records(db.read({
            "animal_type": "Dog",
            "breed": {"$in": ["Doberman Pinscher", "German Shepard", "Golden Retriever", "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20.0, "$lte": 300.0}
        }))
    else:
        df = pd.DataFrame.from_records(db.read({}))

    # Drop the '_id' column if it exists
    if '_id' in df.columns:
        df.drop(columns=['_id'], inplace=True)
        
    # Sort the DataFrame by 'rec_num'
    if 'rec_num' in df.columns:
        df = df.sort_values(by='rec_num')
        
    # Check if the DataFrame is empty
    if df.empty:
        print("No data returned from the database.")
        return [], []  # Return empty data and columns if no records found

    # Define columns for the data table
    columns = [{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]

    # Convert the DataFrame to a dictionary of records
    data = df.to_dict('records')

    return (data, columns)


@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    # Create DataFrame from viewData
    dff = pd.DataFrame.from_dict(viewData)
    
    # Checking DataFrame for breed or empty
    if not dff.empty and 'breed' in dff.columns:
        # Create a pie chart
        figure = px.pie(dff, names='breed')  
        
    else:
        # No data
        figure = px.pie(title='No Data Available')  

    return [
        dcc.Graph(            
            figure=figure
        )    
    ]





@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if selected_columns is None:
        return []
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None or len(viewData) == 0:
        return []
    elif index is None:
        return
    
    if index is None or len(index) == 0:
        row = 0
    else: 
        row = index[0]
    
    dff = pd.DataFrame.from_dict(viewData)
    

        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



app.run_server(debug=True)


Dash app running on http://127.0.0.1:15566/
